In [1]:
import vertexai
import json
import yaml

from vertexai.preview.generative_models import GenerativeModel

In [2]:
vertexai.init(project="generative-ai-teaching", location="us-central1")

In [3]:
model = GenerativeModel("gemini-pro")

In [22]:
output_generation_config={
    "temperature": 0.6,
    "max_output_tokens": 8192,
    "top_p": 0.95
}

# Step 1: Extract News

In [4]:
news_extractor_system_prompt = """
You are tasked to extract key information from news articles.
You will be presented a news article that begins with ###News Article.

Instructions:
Extract the following items from the news article in the input in the following YAML format:
news setting: 
news characters and their main activities: 
news plot summary: 
news information points:  
news plot elements: 

To reiterate, your answer should be in the YAML format specified above.
"""

In [5]:
news_extractor_user_message = """###News Article \n{news_article}"""

In [6]:
news_article_input = """
The US Food and Drug Administration (USFDA) last week approved two breakthrough gene therapies — Casgevy by Vertex Pharmaceuticals and CRISPR Therapeutics, and Lyfgenia by Bluebird Bio — for sickle cell disease (SCD) in patients 12 years and older.

The development marks a milestone medical advancement in treating a debilitating disease that primarily affects red blood cells’ capacity to carry adequate oxygen across the body, with the use of innovative cell-based gene therapies.
Both approved products are made from patients’ own blood stem cells, which are modified, and are given back as a one-time, single-dose infusion as part of a hematopoietic (blood) stem cell transplant.

Casgevy utilises CRISPR/Cas9 (Clustered Regularly Interspaced Short Palindromic Repeats-CRISPR associated) technology, a type of genome editing system.
Emmanuelle Charpentier and Jennifer Doudna were awarded the Nobel Prize in Chemistry in 2020 for discovering CRISPR/Cas9 genetic scissors, called one of the gene technology’s sharpest tools.

In India, which has the highest number of SCD carriers in the world, scientists associated with the Council for Scientific and Industrial Research-Institute of Genomics and Integrative Biology (CSIR-IGIB) have been working since 2018 to develop a gene therapy for SCD using the same technology.

“After showing proof of the therapy developed in human-induced pluripotent stem cells (a particular potent type of stem cell that normally only exists during early embryonic development), we are now in preclinical stage of the therapy’s trial,” Debojyoti Chakraborty, lead scientist of the project at CSIR-IGIB, told ThePrint.

The next step after the animal study, he said, is to start a phase-1 clinical trial for SCD patients in India, in partnership with the All India Institute of Medical Sciences (AIIMS) in Delhi and the department of science and technology after regulatory approvals for the therapy are in place.
Once available in the country, the therapy can be a boon for millions of SCD patients in India which this year saw the launch of the National Sickle Cell Anaemia Elimination Mission — targeting to eliminate the disease by 2047.
"""

In [7]:
prompt_for_news_extraction = (
        news_extractor_system_prompt + 
        news_extractor_user_message.format(
                news_article=news_article_input
        )
)

In [24]:
response = model.generate_content(
    prompt_for_news_extraction,
    generation_config=output_generation_config
)

In [25]:
print(response.text)

news setting: 
- US Food and Drug Administration (USFDA)
- Vertex Pharmaceuticals and CRISPR Therapeutics
- Bluebird Bio
- India (highest number of SCD carriers in the world)
- Council for Scientific and Industrial Research-Institute of Genomics and Integrative Biology (CSIR-IGIB)
- All India Institute of Medical Sciences (AIIMS) in Delhi
- Department of Science and Technology
- India's National Sickle Cell Anaemia Elimination Mission

news characters and their main activities: 
- USFDA: Approved two breakthrough gene therapies for sickle cell disease (SCD)
- Vertex Pharmaceuticals and CRISPR Therapeutics: Developed Casgevy, a gene therapy for SCD using CRISPR/Cas9 technology
- Bluebird Bio: Developed Lyfgenia, another gene therapy for SCD
- Debojyoti Chakraborty: Lead scientist of the SCD gene therapy project at CSIR-IGIB
- Scientists at CSIR-IGIB: Working to develop a gene therapy for SCD using CRISPR/Cas9 technology

news plot summary: 
- USFDA approved two gene therapies, Casgevy a

In [26]:
news_extraction_output = yaml.safe_load(response.text)

In [27]:
news_extraction_output.keys()

dict_keys(['news setting', 'news characters and their main activities', 'news plot summary', 'news information points', 'news plot elements'])

# Step 2: Create Comedic Analogy

In [37]:
comedic_analogy_prompt_template = """
You are tasked to create three comedic analogies based on key information extracted from a news article.
Do not create any offensive or harmful content.
1. List three unique comedic analogies for the situation in the following story:
{news_plot_summary}. Incorporate the following characters only: {news_characters_and_their_main_activities}.
2. Decide the main characters of the news event as two of the most dominant characters in the summary: {news_plot_summary}
3. To act out this analogous premise use the location mentioned here: {news_setting}

Return three analogies in the following YAML format:
comedic analogy 1: <first analogy>,
comedic analogy 2: <second analogy>,
comedic analogy 3: <third analogy>
Do not return any other output except this YAML.
"""

In [38]:
prompt_for_comedic_analogy = comedic_analogy_prompt_template.format(
    news_plot_summary=news_extraction_output['news plot summary'],
    news_characters_and_their_main_activities=news_extraction_output['news characters and their main activities'],
    news_setting=news_extraction_output['news setting']
)

In [39]:
print(prompt_for_comedic_analogy)


You are tasked to create three comedic analogies based on key information extracted from a news article.
Do not create any offensive or harmful content.
1. List three unique comedic analogies for the situation in the following story:
['USFDA approved two gene therapies, Casgevy and Lyfgenia, for sickle cell disease (SCD) in patients 12 years and older', 'Both therapies use patients’ own blood stem cells, which are modified and given back as a one-time, single-dose infusion as part of a hematopoietic stem cell transplant', 'In India, scientists at CSIR-IGIB are working on developing a gene therapy for SCD using the same technology', 'The therapy is in the preclinical stage of trials and is expected to start phase-1 clinical trials in India in partnership with AIIMS and the department of science and technology']. Incorporate the following characters only: [{'USFDA': 'Approved two breakthrough gene therapies for sickle cell disease (SCD)'}, {'Vertex Pharmaceuticals and CRISPR Therapeutic

In [40]:
response = model.generate_content(
    prompt_for_comedic_analogy,
    generation_config=output_generation_config
)

In [42]:
print(response.text)

comedic analogy 1: The USFDA approving two gene therapies for sickle cell disease is like a superhero team-up between CRISPR Therapeutics and Bluebird Bio, saving the day for patients worldwide.
comedic analogy 2: Debojyoti Chakraborty and the scientists at CSIR-IGIB are like culinary masters, cooking up a gene therapy for sickle cell disease using the CRISPR/Cas9 technology as their secret ingredient.
comedic analogy 3: The race to develop a cure for sickle cell disease is like a friendly competition between the US and India, with the USFDA and CSIR-IGIB leading the charge like rival sports teams battling it out on the field of medical innovation.


In [43]:
comedic_analogies = yaml.safe_load(response.text)

In [44]:
comedic_analogies.keys()

dict_keys(['comedic analogy 1', 'comedic analogy 2', 'comedic analogy 3'])

# Step 3: Create Script

In [46]:
comedic_script_prompt_template = """
You are tasked to create a comedic skit.
Write a script for a comedy skit about: {script_plot}. Cover the following information: {news_information_points}.
The characters should be exactly: {news_characters_and_their_main_activities}.
It should be set in {news_setting}. It should be entertaining.
The dialogue should be colloquial and engaging. The dialogue should be 10 to 12 lines long.
Each line of dialogue should be short - less than 20 words. End it with a punchline.
"""

In [49]:
prompt_for_comedic_script = comedic_script_prompt_template.format(
    script_plot=comedic_analogies['comedic analogy 1'],
    news_information_points=news_extraction_output['news information points'],
    news_characters_and_their_main_activities=news_extraction_output['news characters and their main activities'],
    news_setting=news_extraction_output['news setting']
)

In [51]:
response = model.generate_content(
    prompt_for_comedic_script,
    generation_config=output_generation_config
)

In [54]:
comedic_script = response.text

In [55]:
output_file_name = 'gemini-pro_comedic-script.txt'

In [56]:
with open(output_file_name, 'w') as f:
    f.writelines(comedic_script)